In [157]:
import os
os.environ["TMPDIR"] = "/tmp"  # set the folder for temporary files

from qiskit.opflow.gradients import Gradient, NaturalGradient
from qiskit_mod.qiskit_ter import LinCombFullmod,LinCombMod
from qiskit_nature.algorithms import VQEUCCFactory,GroundStateEigensolver
from qiskit.algorithms.optimizers import SLSQP,L_BFGS_B,SPSA, COBYLA
from qiskit_nature.circuit.library import HartreeFock
from qiskit.circuit.library import TwoLocal
from qiskit_nature.mappers.second_quantization import JordanWignerMapper, ParityMapper
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.drivers.second_quantization.pyscfd import PySCFDriver
from qiskit_nature.drivers import UnitsType, Molecule
from qiskit_nature.problems.second_quantization import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import FreezeCoreTransformer
from qiskit import Aer
from qiskit.utils import QuantumInstance

In [158]:
molecule = 'H .0 {0} {0}; H .0 {0} -{0};'
orbitals_to_remove = []
basis = 'sto3g'
optimizer = L_BFGS_B(maxiter=1000)
qubit_converter = QubitConverter(mapper=ParityMapper(), two_qubit_reduction=True) 
molecule_set = molecule.format(1/2)
driver = PySCFDriver(atom=molecule_set,
                        unit=UnitsType.ANGSTROM,
                        basis=basis)
#qmolecule = driver.run()
es_problem = ElectronicStructureProblem(driver,
                                        q_molecule_transformers = [FreezeCoreTransformer(freeze_core = True,
                                                                                        remove_orbitals = orbitals_to_remove
                                                                                        )]
                                        )
second_q_op = es_problem.second_q_ops()
qubit_op = qubit_converter.convert(second_q_op[0],
                                    num_particles = es_problem.num_particles) #this line is useful for initialize the converter and get the right hartree-fock number of qubits
particle_number = es_problem.properties_transformed.get_property("ParticleNumber")

In [401]:
from random import seed


shots = 1024
qiskitbackend = Aer.get_backend('statevector_simulator')
quantum_instance = QuantumInstance(backend=qiskitbackend,seed_simulator=42, seed_transpiler=42, shots=10000)
from qat.qpus import PyLinalg
qlm_qpu = PyLinalg()

In [402]:
from qiskit_mod.wrapper2myqlm import *
from qiskit_mod.my_junction import IterativeExplorationVQE,get_energy_evaluation_QLM
VQE.get_energy_evaluation = get_energy_evaluation_QLM #override the function, class-wide
plugin_in_use = IterativeExplorationVQE

In [988]:
from qiskit.circuit import ParameterExpression, ParameterVector, Parameter
from typing import Callable, Iterable, List, Optional, Tuple, Union,  cast ,Dict
from qiskit.providers import BaseBackend, Backend
from qiskit.opflow import ExpectationBase, CircuitSampler, PauliExpectation, OperatorStateFn, CircuitStateFn, PauliOp, ListOp, SummedOp,ComposedOp
from qiskit.quantum_info import Pauli
LIST_OF_GATES = ['i', 'id', 'iden', 'u', 'u0', 'u1', 'u2', 'u3', 'x', 'y', 'z', 'h', 's', 'sdg', 't', 'tdg', 'rx', 'ry', 'rz', 'cx', 'cy', 'cz', 'ch', 'crz', 'cu1', 'cu3', 'swap', 'ccx', 'cswap', 'r']

def extract_and_compute_circ(self, operator: OperatorBase,prev_coeff=1) -> None:
            """
            Recursively extract the ``CircuitStateFns`` contained in operator into the
            ``_circuit_ops_cache`` field.
            """
                # print(operator,'vs',operator.coeff)
            #print('Coeff',operator.coeff, type(operator))
            if isinstance(operator, ComposedOp):
               
                coefficient = prev_coeff*operator.coeff
                for i,op in enumerate(operator.oplist):
                    if isinstance(op,CircuitStateFn):
                        circuit = op.to_circuit_op()
                        #coefficient*=op.coeff
                    elif isinstance(op,OperatorStateFn):
                        operator_to_run = op
                        #coefficient*=op.coeff
                    elif isinstance(op,ListOp):
                        for op2 in op.oplist:
                            if isinstance(op2,CircuitStateFn):
                                circuit = op2.to_circuit_op()
                                #coefficient*=op2.coeff
                if circuit is not None and operator_to_run is not None:
                    value = run_circuit_in_QLM(self,circuit,operator_to_run, coefficient)
                    #print('Computed',value, 'with coefficient',coefficient)
                return value
                
            elif isinstance(operator, SummedOp):
                list_of_values=[]
                coefficient = prev_coeff*operator.coeff
                for i,op in enumerate(operator.oplist):
                    list_of_values.append(extract_and_compute_circ(self, op,prev_coeff=coefficient))
                summation= sum(list_of_values)
                print('Computed sum',summation)
                return summation

            elif isinstance(operator, ListOp):
                list_of_values=[]
                
                coefficient = prev_coeff*operator.coeff
                print(coefficient)
                for i,op in enumerate(operator.oplist):
                    list_of_values.append(extract_and_compute_circ(self, op, prev_coeff=coefficient))
                return list_of_values



def run_circuit_in_QLM(self,circuit,operator, coefficient):

    operator_pauli = operator._primitive.to_pauli_op()
    operator_meas_inv = PauliOp(primitive= Pauli(operator_pauli.primitive.to_label()[::-1]), coeff=operator_pauli.coeff)
    
    #result.append(create_and_run_job(self,operat,p_values_dict,self.list_of_circuits[k]))
    transpiled_circ = transpile(circuit.to_circuit(),
                            basis_gates=LIST_OF_GATES,
                            optimization_level=0)
    qcirc = qiskit_to_qlm(transpiled_circ)
    job = qcirc.to_job(observable=Observable(operator_meas_inv.num_qubits, matrix=operator_meas_inv.to_matrix()),nbshots=self.nb_shots) 
    result_temp = self.execute(job)
    return coefficient * np.real(result_temp.value)




def gradient_wrapper_for_QLM(
        self,
        operator: OperatorBase,
        bind_params: Union[ParameterExpression, ParameterVector, List[ParameterExpression]],
        grad_params: Optional[
            Union[
                ParameterExpression,
                ParameterVector,
                List[ParameterExpression],
                Tuple[ParameterExpression, ParameterExpression],
                List[Tuple[ParameterExpression, ParameterExpression]],
            ]
        ] = None,
        backend: Optional[Union[BaseBackend, Backend, QuantumInstance]] = None,
        expectation: Optional[ExpectationBase] = None,
    ) -> Callable[[Iterable], np.ndarray]:
        """Get a callable function which provides the respective gradient, Hessian or QFI for given
        parameter values. This callable can be used as gradient function for optimizers.

        Args:
            operator: The operator for which we want to get the gradient, Hessian or QFI.
            bind_params: The operator parameters to which the parameter values are assigned.
            grad_params: The parameters with respect to which we are taking the gradient, Hessian
                or QFI. If grad_params = None, then grad_params = bind_params
            backend: The quantum backend or QuantumInstance to use to evaluate the gradient,
                Hessian or QFI.
            expectation: The expectation converter to be used. If none is set then
                `PauliExpectation()` is used.

        Returns:
            Function to compute a gradient, Hessian or QFI. The function
            takes an iterable as argument which holds the parameter values.
        """
        from qiskit.opflow.converters import CircuitSampler


        if not grad_params:
            grad_params = bind_params

        grad = self.convert(operator, grad_params)
        if expectation is None:
            expectation = PauliExpectation()
        #grad = expectation.convert(grad)
        print('General operator',operator)
        print('bind_params',bind_params)

        # extract_and_compute_circ(self,grad)
        # print('List of ansatz',len(self.list_of_circuits))


        def gradient_fn(p_values):
            p_values = [0.324, 0.54, 1.2, 0.99]
            print('Start gradient')
            p_values_dict = dict(zip(bind_params, p_values))
            if not backend:
                converter = grad.assign_parameters(p_values_dict)
                return np.real(converter.eval())
            else:
                p_values_dict = {k: [v] for k, v in p_values_dict.items()} # remake the dict with list of values
                # converter = grad.assign_parameters(p_values_dict) # assign the values to the parameters
                self.list_of_circuits = []
                self.list_of_operators = []
                self.list_of_coefficients = []
                #print(converter)
                converted_circ = expectation.convert(grad)
                converter = grad.assign_parameters(p_values_dict) # assign the values to the parameters
                # print('Circ with assigned values:')
                # print(converter)
                dictio = extract_and_compute_circ(self,converter)
                print('#########################################################')
            
                    
                print('LIST',dictio[0], dictio[1])
                val = dictio[1]
                matrix = np.zeros((len(val),len(val)))
                upper_w_diag = np.triu_indices(len(val))
                matrix[upper_w_diag] = np.array(np.concatenate(val))
                i_upper = np.triu_indices(len(val), 1)
                i_lower = np.tril_indices(len(val), -1)
                matrix[i_lower] = matrix[i_upper]

                

                result = []
                #print('p_values_dict',p_values_dict)
                converter = CircuitSampler(backend=backend).convert(converted_circ, p_values_dict)
                QLM_result = converter[0].combo_fn([dictio[0],1/4*matrix])
                print('There we go',QLM_result)

                # for k,op in enumerate(self.list_of_operators):
                #     circuit = self.list_of_circuits[k]
                #     operator_pauli = op._primitive.to_pauli_op()
                #     operator_meas = PauliOp(primitive= Pauli(operator_pauli.primitive.to_label()[::-1]), coeff=operator_pauli.coeff)
                    
                #     #result.append(create_and_run_job(self,operat,p_values_dict,self.list_of_circuits[k]))
                #     transpiled_circ = transpile(circuit.to_circuit(),
                #                             basis_gates=LIST_OF_GATES,
                #                             optimization_level=0)
                #     qcirc = qiskit_to_qlm(transpiled_circ)
                #     job = qcirc.to_job(observable=Observable(operator_meas.num_qubits, matrix=operator_meas.to_matrix()),nbshots=self.nb_shots) 
                #     result_temp = self.execute(job)
                #     result.append(self.list_of_coefficients[k]*np.real(result_temp.value))
                #     if k<1 and True:
                #         print('Original op',op, 'Flipped',operator_meas ,'coef of original',operator_pauli.coeff,'coef of flipped',operator_meas.coeff)
                #         print('Coeff',self.list_of_coefficients[k])
                        
                #         print(circuit.to_circuit())
                #         print()
                #         print('QLM:',self.list_of_coefficients[k]*np.real(result_temp.value))
                #         print('Qiskit',converter[0].eval())
                #         #print('Qiskit',converter[0][0][0][0].eval()+converter[0][0][0][1].eval()+converter[0][0][0][2].eval()+converter[0][0][0][3].eval()+converter[0][0][0][4].eval())
                #         print('single',converter[0]._state())
                
                
                print('HERE!@@@@@@@@')
                # print([op.eval() for op in converter[0][0][0].oplist])
                # #print(converter,p_values_dict)
                # print('Original result',np.real(converter[0][0].eval()))
                print('Original result',np.real(converter.eval()))
                # print('Mine',len(result),result)
                # print('Mine',sum(result[0:5]),sum(result[5:10]),sum(result[10:15]),sum(result[15:20]),)
                # print('last ten',result[20:])
                #return np.real(converter.eval()[0])
                return QLM_result
                #return np.asarray([sum(result[0:5]),sum(result[5:10]),sum(result[10:15]),sum(result[15:20])])
                

        return gradient_fn

In [989]:
#gradient
from qiskit.opflow.gradients import DerivativeBase

DerivativeBase.gradient_wrapper = gradient_wrapper_for_QLM

In [990]:
nat_grad = NaturalGradient(grad_method=LinCombMod(img=False),
                               qfi_method=LinCombFullmod(),
                               regularization='ridge')
hartree_fock_state = HartreeFock(particle_number.num_spin_orbitals,
                                    (particle_number.num_alpha, particle_number.num_beta),
                                    qubit_converter)
num_qubits = hartree_fock_state.num_qubits
ansz = TwoLocal(hartree_fock_state.num_qubits,
                    rotation_blocks=['ry'], 
                    entanglement_blocks='cx',
                    entanglement="full",
                    reps=1,
                    initial_state=hartree_fock_state)
Im_solver = VQEUCCFactory(quantum_instance,
                            gradient=nat_grad,
                            optimizer=L_BFGS_B(maxiter=100), # good optimizer for Nat. Grad.
                            ansatz=ansz.decompose())



In [991]:
calcIevo = GroundStateEigensolver(qubit_converter,
                                    Im_solver)
stack = build_QLM_stack(calcIevo,molecule_set,
                        plugin_in_use,
                        qlm_qpu,
                        shots = 10000,
                        remove_orbitals = orbitals_to_remove)
if True:
    resIevo = run_QLM_stack(stack,num_qubits,10000)
    # resIevo = calcIevo.solve(es_problem)

    print('Imaginary time evo. ,',
            resIevo.total_energies, ',exec_time=',
            len(resIevo.raw_result.optimal_parameters))

10000
General operator ComposedOp([
  OperatorMeasurement((-1.0692434904119297-8.326672684688674e-17j) * II
  - 0.267528649942086 * ZI
  + (0.2675286499420861-1.3877787807814457e-17j) * IZ
  + (-0.009014930058166226-1.3877787807814457e-17j) * ZZ
  + (0.19679058348547016+3.469446951953614e-18j) * XX),
  CircuitStateFn(
          ┌───┐    ┌──────────┐     ┌──────────┐
  q_0: ───┤ X ├────┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├
       ┌──┴───┴───┐└──────────┘┌─┴─┐├──────────┤
  q_1: ┤ Ry(θ[1]) ├────────────┤ X ├┤ Ry(θ[3]) ├
       └──────────┘            └───┘└──────────┘
  )
])
bind_params [ParameterVectorElement(θ[0]), ParameterVectorElement(θ[1]), ParameterVectorElement(θ[2]), ParameterVectorElement(θ[3])]
Start gradient
1.0
1.0
Computed sum 0.09879583042125992
Computed sum 0.026414802077487393
Computed sum 0.32459113799903705
Computed sum 0.07658634676847582
0.25
0.25
Computed sum 0.9999999999999996
Computed sum 0.0
Computed sum 0.5141359916531129
Computed sum 0.0
0.25
Computed sum 0.99999999999

KeyboardInterrupt: 

-1.10115

In [ ]:
from qiskit.opflow import (StateFn, Zero, One, Plus, Minus, H,
                           DictStateFn, VectorStateFn, CircuitStateFn, OperatorStateFn)
from qiskit.opflow import I, X, Y, Z 


opera =((-1.0692434904119297-8.326672684688674e-17j) * I^I) #- (0.267528649942086 * Z^I)\
#   +( (0.2675286499420861-1.3877787807814457e-17j) * I^Z)\
#   + ((-0.009014930058166226-1.3877787807814457e-17j) * Z^Z)\
#   + ((0.19679058348547016+3.469446951953614e-18j) * X^X)


In [ ]:
gradf = calcIevo.solver.gradient.gradient_wrapper(~StateFn(opera) @ StateFn(ansz.decompose()),list(ansz.parameters), backend = quantum_instance)

operator ComposedOp([
  OperatorMeasurement((-1.0692434904119297+8.326672684688674e-17j) * II),
  CircuitStateFn(
          ┌───┐    ┌──────────┐     ┌──────────┐
  q_0: ───┤ X ├────┤ Ry(θ[0]) ├──■──┤ Ry(θ[2]) ├
       ┌──┴───┴───┐└──────────┘┌─┴─┐├──────────┤
  q_1: ┤ Ry(θ[1]) ├────────────┤ X ├┤ Ry(θ[3]) ├
       └──────────┘            └───┘└──────────┘
  )
])
bind_params [ParameterVectorElement(θ[0]), ParameterVectorElement(θ[1]), ParameterVectorElement(θ[2]), ParameterVectorElement(θ[3])]


In [ ]:
conv =gradf([0.324, 0.54, 1.2, 9.99])

P values  [0.324, 0.54, 1.2, 9.99]
p_values_dict {ParameterVectorElement(θ[0]): [0.324], ParameterVectorElement(θ[1]): [0.54], ParameterVectorElement(θ[2]): [1.2], ParameterVectorElement(θ[3]): [9.99]}
Original op OperatorMeasurement((2+0j) * ZII) Flipped (2+0j) * IIZ coef of original (2+0j) coef of flipped (2+0j)
Coeff (-1.0692434904119297+8.326672684688674e-17j) , Value -1.1102230246251565e-16
                     ┌───┐                 ┌───┐┌───────────────────────┐     »
      q_0: ──────────┤ X ├─────────────────┤ Y ├┤ Ry(0.324000000000000) ├──■──»
           ┌─────────┴───┴─────────┐       └─┬─┘└───────────────────────┘┌─┴─┐»
      q_1: ┤ Ry(0.540000000000000) ├─────────┼───────────────────────────┤ X ├»
           └─────────┬───┬─────────┘┌─────┐  │            ┌───┐          └───┘»
q185195_0: ──────────┤ H ├──────────┤ Sdg ├──■────────────┤ H ├───────────────»
                     └───┘          └─────┘               └───┘               »
«           ┌──────────────────────┐
«   

IndexError: list index out of range

In [ ]:
mat = np.asarray([[0.9999999999999996, 0.0, 0.5141359916531129, 0.0][0.0,0.9999999999999994,0.0, -0.9479695613221306], [0.9999999999999996, 0.27306097260886564], [0.9999999999999997]])
#mat = np.asarray([ [0.9999999999999997,0,0,0],[0.9999999999999996, 0.27306097260886564,0,0],[0.9999999999999994, 0.0, -0.9479695613221306,0],[0.9999999999999996, 0.0, 0.5141359916531129, 0.0]])


/tmp/ipykernel_7622/1101136237.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  mat = np.asarray([[0.9999999999999996, 0.0, 0.5141359916531129, 0.0,0.9999999999999994], [0.0, -0.9479695613221306], [0.9999999999999996, 0.27306097260886564], [0.9999999999999997]])


In [ ]:
upper_w_diag = np.triu_indices(4)
i_upper = np.triu_indices(4, 1)
i_lower = np.tril_indices(4, -1)
matrix = np.zeros((4,4))
valus = [0.9999999999999996, 0.0, 0.5141359916531129, 0.0, 0.9999999999999994, 0.0, -0.9479695613221306, 0.9999999999999996, 0.27306097260886564, 0.9999999999999997]

matrix[upper_w_diag] = valus
matrix[i_lower] = matrix[i_upper]

conv[0].combo_fn([[0.09879583, 0.0264148,  0.32459114, 0.07658635],1/4*matrix])

array([-0.27994812,  0.32068078,  1.32423912,  0.24095422])

In [ ]:

val = [[0.9999999999999996, 0.0, 0.5141359916531129, 0.0], [0.9999999999999994, 0.0, -0.9479695613221306], [0.9999999999999996, 0.27306097260886564], [0.9999999999999997]]


matrix = np.zeros((4,4))
matrix[upper_w_diag] = np.asarray(valus)
matrix

array([[ 1.        ,  0.        ,  0.51413599,  0.        ],
       [ 0.        ,  1.        ,  0.        , -0.94796956],
       [ 0.        ,  0.        ,  1.        ,  0.27306097],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [ ]:

matrix = np.zeros((len(val),len(val)))
upper_w_diag = np.triu_indices(len(val))
matrix[upper_w_diag] = np.array(np.concatenate(val))
i_upper = np.triu_indices(len(val), 1)
i_lower = np.tril_indices(len(val), -1)
matrix[i_lower] = matrix[i_upper]
print(matrix)

[[ 1.          0.          0.51413599  0.        ]
 [ 0.          1.          0.         -0.94796956]
 [ 0.51413599  0.          1.          0.27306097]
 [ 0.         -0.94796956  0.27306097  1.        ]]
